# Ciencia De Datos Aplicada (ITBA): Segundo Entregable

## Recopilación y Preparación de Datos

**Equipo:** Uriel Arias - 63504, Cristian Tepedino - 62830

**Nombre del proyecto**: Sistema de predicción de resultados de
carreras de Fórmula 1

## 🧾 1. Importación y carga de librerías

In [3]:
!pip install fastf1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 5.4 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [21]:
import os

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import fastf1


cache_dir = './cache'
os.makedirs(cache_dir, exist_ok=True)

fastf1.Cache.enable_cache(cache_dir)

sns.set(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)

import warnings
warnings.filterwarnings("ignore")

## 🧾 2. Carga de datos

In [ ]:
# Pongamos de ejemplo el 'Italian Grand Prix' que se celebra en el Autodromo Nazionale di Monza. Vamos a elegir la data correspondiente a Charles Leclerc respecto de la edición de 2024 de este Gran Premio.

# Cargamos cada una de las sesiones con toda la información de la sesión
fp1 = fastf1.get_session(2024, 'Italian Grand Prix', 'FP1')
fp1.load(weather=True)

fp1_car_telemetry = fp1.car_data['16'] # elegimos quedarnos con la de CL (#16)
fp1_weather = fp1.weather_data

fp2 = fastf1.get_session(2024, 'Italian Grand Prix', 'FP2')
fp2.load(weather=True)

fp2_car_telemetry = fp2.car_data['16'] # elegimos quedarnos con la de CL (#16)
fp2_weather = fp2.weather_data

fp3 = fastf1.get_session(2024, 'Italian Grand Prix', 'FP3')
fp3.load()

fp3_car_telemetry = fp3.car_data['16'] # elegimos quedarnos con la de CL (#16)
fp3_weather = fp3.weather_data

qualy = fastf1.get_session(2024, 'Italian Grand Prix', 'Q')
qualy.load()

qualy_car_telemetry = qualy.car_data['16'] # elegimos quedarnos con la de CL (#16)
qualy_weather = qualy.weather_data

race = fastf1.get_session(2024, 'Italian Grand Prix', 'R')
race.load()

# ahora cargamos los resultados de la carrera
race_results = race.results # df
# Cargamos la información del circuito
circuit_info = race.session_info # dict

core           INFO 	Loading data for Italian Grand Prix - Qualifying [v3.6.1]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['77', '44', '33', '4', '3', '10', '55', '16', '11', '99', '5', '18', '14', '31', '63', '6', '22', '47', '88', '9']


## 🗒️ 3. Descripción del dataset
Este dataset obtenido de la API de FastF1 contiene información de cada sesión de F1 así como de los circuitos. Para las sesiones se usa el objeto `Session` que permite obtener la información de 'FP1', 'FP2', 'FP3', 'Qualifying' y 'Race' detallada con tiempos y telemetría por vuelta para todos los pilotos que hayan participado de la misma. A su vez, se resumen los resultados de un Grand Prix en el objeto `Results` donde se muestra para un piloto los mejores tiempos de cada sesión y otros datos estadísticos relevantes.  Por otro lado, se brinda información 
### Campos por sesión (`Session`)
- `name`: nombre de la sesión (ej. "Race", "Qualifying").
- `event`: objeto con datos del evento (país, nombre del GP, año, ronda).
- `date`: fecha y hora oficial de inicio de la sesión.
- `weather_data`: dataframe con condiciones climáticas registradas (`WeatherData`).
- `track_status`: estados de pista durante la sesión (banderas, SC, VSC, etc.).
- `laps`: dataframe con todas las vueltas (`Lap`).
- `results`: dataframe con resultados finales de pilotos/equipos (`Results`).
- `car_data`: telemetría cruda por piloto (`CarData`).
- `pos_data`: datos de posición en pista (coordenadas GPS, estado).
- `drivers`: lista de códigos de pilotos participantes.
- `f1_api_support`: booleano, indica si hay datos disponibles de la API oficial.
- `session_info`: información general (tipo de sesión, duración, etc.).

### Campos por vuelta (`Lap`)
- `Driver`: código del piloto (ej. VER, HAM).
- `DriverNumber`: número oficial del coche (ej. 33, 44).
- `Team`: nombre del equipo.
- `LapNumber`: número de vuelta en la sesión.
- `Stint`: índice de stint al que pertenece la vuelta.
- `LapStartTime`: tiempo absoluto de inicio de la vuelta (relativo a inicio de sesión).
-  `LapTime`: tiempo total de la vuelta (Timedelta).
- `Sector1Time`, `Sector2Time`, `Sector3Time`: tiempos por sector.
- `Sector1SessionTime`, `Sector2SessionTime`, `Sector3SessionTime`: marca de tiempo en la sesión cuando termina cada sector.
- `Time`: tiempo de paso por la línea de meta (relativo a inicio de sesión).
- `PitInTime`, `PitOutTime`: tiempos en que entró/salió de pits.
- `Compound`: tipo de compuesto (SOFT, MEDIUM, HARD, INTERMEDIATE, WET).
- `TyreLife`: número de vueltas que llevaba el neumático.
- `FreshTyre`: booleano, si era nuevo o no.
- `LapStartStint`: si la vuelta marca el inicio de un stint.
- `IsAccurate`: indica si los datos son completos/fiables.
- `IsPersonalBest`: si es la mejor vuelta personal del piloto.
- `TrackStatus`: estado de pista (bandera verde, amarilla, roja, SC, VSC, etc.).
- `Position`: posición del piloto al terminar la vuelta.
- `Deleted`: si la vuelta fue eliminada por dirección de carrera.
- `DeletedReason`: motivo de la eliminación (ej. track limits).
-  `LapStartDate`:  marca de tiempo absoluta (datetime).
### Campos de datos de coche (`CarData`)
- `Time`: timestamp en la sesión.
- `RPM`: revoluciones del motor.
- `Speed`: velocidad del coche (km/h).
- `nGear`: marcha engranada.
- `Throttle`: % de acelerador.
- `Brake`: % de frenada.
- `DRS`: estado del DRS.

### Campos de clima (`WeatherData`)
- `Time`: timestamp en la sesión.
- `AirTemp`: temperatura del aire (°C).
- `TrackTemp`: temperatura de pista (°C).
- `Humidity`: humedad relativa (%).
- `Pressure`: presión atmosférica (hPa).
- `Rainfall`: lluvia (0/1).
- `WindSpeed`: velocidad del viento (m/s).
- `WindDirection`: dirección del viento (grados).
### Campos de resultados (`Results`)
Cada fila corresponde a un piloto en la sesión.

- `Abbreviation`: código de piloto (ej. VER, HAM).
- `DriverNumber`: número oficial del coche.
- `BroadcastName`: nombre mostrado en TV.
- `TeamName`: equipo.
- `Position`: posición final en la sesión.
- `GridPosition`: posición de largada (carrera).
- `Status`: estado final (Finished, DNF, DSQ…).
- `Points`: puntos sumados.
- `Time`: tiempo total de carrera (si aplica).
- `Q1`, `Q2`, `Q3`: tiempos de clasificación por ronda (en qualifying).
- `FastestLapTime`: mejor tiempo de vuelta en la sesión.
- `FastestLapSpeed`: velocidad máxima en esa vuelta rápida.
- `BestLapNumber`: número de la vuelta más rápida.

### Campos de circuito (`SessionInfo`)
- `CircuitName`: nombre oficial del circuito.
- `Location`: ciudad donde se ubica.
- `Country`: país del circuito.
- `OfficialEventName`: nombre oficial del GP (ej. "FORMULA 1 GRAN PREMIO DE ESPAÑA 2024").
- `EventDate`: fecha del GP.
- `RoundNumber`: número de ronda en el campeonato.
- `Session1`, `Session2`, … `Session5`: horarios de prácticas, quali, sprint y carrera.
- `CircuitKey`: identificador único del circuito en la API.
- `EventFormat`: formato del fin de semana (estándar, sprint, etc.).

### 🔍 4. Análisis exploratorio de datos (EDA)

### 🧼 5. Diagnóstico de calidad de datos


### 🧪 6. Transformaciones realizadas

**Variables principales:**
- Driver:	Código del piloto
- DriverName: Nombre del piloto
- LapNumber:	Número de vúelta
- LapTime:	Tiempo total de la vuelta (en segundos)
- Speed_mean:	Velocidad promedio durante la vuelta (km/h)
- Throttle_mean: Porcentaje promedio de acelerador durante la vuelta
- Brake_mean:	Porcentaje promedio de freno durante la vuelta
- RPM_mean:	Revoluciones promedio del motor durante la vuelta
- DRS_max:	Valor máximo del DRS en la vuelta (0 = cerrado, 1 = abierto)
- TyreLife: Cantidad de vueltas de uso del neumático (incluyendo el acumulado por sesiones anteriores).
- Compound: Nombre del compuesto a usar (SOFT, MEDIUM, HARD, INTERMEDIATE, WET). No se distingue entre compuestos de C1 a C5.
- FreshTyre: Indica si es un neumático nuevo (especialmente útil para comparar en vueltas de clasificación).
- TrackStatus: Indica el estado de la pista al momento de registrarse una vuelta (valores del 1-7 indicando pista libre, banderas amarillas, safety cars, banderas rojas e inicio y fin de virtual safety cars)

**Variable objetivo potencial:**
- LapTime: regresión para predecir el tiempo de vuelta de un piloto.
- Position: clasificación del ganador/top3/top10 por vuelta o por sesión

**Notas:**
- Cada fila corresponde a una vuelta de un piloto.
- La telemetría se ha agregado usando estadísticas por vuelta (media o máximo) para reducir el número de filas y simplificar el modelado.
- Las variables de clima son promedios de toda la sesión, y pueden servir como features adicionales para mejorar la predicción del desempeño.


In [ ]:
#Cargar sesión de ejemplo
session = fastf1.get_session(2021, 'Italian Grand Prix', 'R')
session.load()

#Obtener laps y filtrar vueltas rápidas válidas
laps = session.laps
laps = laps.pick_quicklaps()

#Obtener telemetría por piloto
telemetry_features = []

for driver in session.drivers:
    driver_laps = laps.pick_driver(driver)
    driver_info = session.get_driver(driver)
    driver_name = f"{driver_info['FirstName']} {driver_info['LastName']}"

    for _, lap in driver_laps.iterlaps():
        lap_telemetry = lap.get_telemetry()
        telemetry_features.append({
            'Driver': driver,
            'DriverName': driver_name,
            'LapNumber': lap.LapNumber,
            'LapTime': lap.LapTime.total_seconds(),
            'Speed_mean': lap_telemetry['Speed'].mean(),
            'Throttle_mean': lap_telemetry['Throttle'].mean(),
            'Brake_mean': lap_telemetry['Brake'].mean(),
            'RPM_mean': lap_telemetry['RPM'].mean(),
            'DRS_max': lap_telemetry['DRS'].max()
        })

df_telemetry = pd.DataFrame(telemetry_features)

#Agregar datos de clima
weather = session.weather_data
weather_summary = weather.mean().to_dict()

# Agregamos clima a cada fila del DataFrame
for key, value in weather_summary.items():
    df_telemetry[key] = value

#Revisar el DataFrame final
df_telemetry.head()

### ✍️ 7. Reflexión final
